In [2]:
import pandas as pd
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [45]:
!python -m spacy download ru_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 6.7 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [3]:
path_dict = {
    'train': 'data/Соответствия.xlsx',
    'etalon': 'data/Эталонная номенклатура.xlsx',
}


In [4]:
train = pd.read_excel(path_dict['train'])


In [5]:
etalon = pd.read_excel(path_dict['etalon'])


In [6]:
N = 1000
train = train.iloc[:N]
etalon = etalon.iloc[:N]


In [15]:
train.columns

Index(['Описание', 'Эталонная номенклатура'], dtype='object')

In [16]:
#save to csv
train.to_csv('data/train1000.csv', index=False)

In [7]:
train

,Описание,Эталонная номенклатура
0,Болт фундаментный 2.1 М24х800-09Г2С-6 ГОСТ 243...,Болт 1.1.М24х800 ст.09Г2С ГОСТ 24379.1-2012
1,А1 Болт 1.1 М36х1400 ГОСТ 24379.1-2012 GCC-LGN...,Болт 1.1.М36х1400 ГОСТ 24379.1-2012 оцинк.
2,Болт 1.2.М12x500 ГОСТ 24379.1-2012 Сборный ГОС...,Болт 1.2.М12х500 ст.сб ГОСТ 24379.1-2012
3,Болт 1.2.МЗ0х900 ГОСТ 24379.1-2012 Сборный ГОС...,Болт 1.2.М30х900 ст.сб ГОСТ 24379.1-2012
4,Болт 2.1.М24х450 ст.09Г2С-6 ГОСТ 24379.1-80 в ...,Болт 2.1.М24х450 ст.09Г2С-6 ГОСТ 24379.1-2012
...,...,...
995,Гайка шестигранная нормальная М16-5 ГОСТ ISO 4...,Гайка М16-5 ГОСТ ИСО 4032-2014
996,Гайка М12.016 ГОСТ Р ИСО 4032-2014 20 ГОСТ 105...,Гайка М12.016 ст.20 ГОСТ ИСО 4032-2014
997,Гайка М12.016 ГОСТ Р ИСО 4032-2014 20 ГОСТ 105...,Гайка М12.016 ст.20 ГОСТ ИСО 4032-2014
998,Гайка М12.016 ГОСТ Р ИСО 4032-2014 20 ГОСТ 105...,Гайка М12.016 ст.20 ГОСТ ИСО 4032-2014


In [9]:
train.columns

Index(['Описание', 'Эталонная номенклатура'], dtype='object')

In [8]:
etalon

,Номенклатура
0,AS17-L100x50x10-30-CS3
1,"Hardtop XP Std Comp A RAL 6011 Jotun 17,33л"
2,"Hardtop XP Std Comp B Jotun 1,8л"
3,"Hardtop XP Std Comp B RAL 1023 Jotun 1,8л"
4,"Hardtop XP Std Comp B RAL 6011 Jotun 1,8л."
...,...
995,"Бобышка БП1-М20х1,5-100 ст.10Х17Н13М2Т ТУ 36-1..."
996,"Бобышка БП1-М20х1,5-100 ст.13ХФА ТУ 4218-032-0..."
997,"Бобышка БП1-М20х1,5-100 ст.20 ТУ 36-1097-85"
998,"Бобышка БП1-М20х1,5-100 ст.20 УХЛ ТУ 4218-032-..."


In [51]:
# !pip install name_matching


In [10]:
import pandas as pd
from name_matching.name_matcher import NameMatcher

etalon_column_name = 'Эталонная номенклатура'



# initialise the name matcher
matcher = NameMatcher(number_of_matches=1,
                      legal_suffixes=True,
                      common_words=True,
                      top_n=50,
                      verbose=True)

# adjust the distance metrics to use
matcher.set_distance_metrics(['bag', 'typo', 'refined_soundex'])

# load the data to which the names should be matched
matcher.load_and_process_master_data(column=etalon_column_name,
                                     df_matching_data=train,
                                     transform=True)

# perform the name matching on the data you want matched
matches = matcher.match_names(to_be_matched=train,
                              column_matching='Описание')



preprocessing...

preprocessing complete 
 searching for matches...



100%|██████████| 1/1 [00:00<00:00, 27.00it/s]


possible matches found   
 fuzzy matching...



100%|██████████| 1000/1000 [01:30<00:00, 11.08it/s]

done


In [53]:
train

,Описание,Эталонная номенклатура
0,Болт фундаментный 2.1 М24х800-09Г2С-6 ГОСТ 243...,1124800 092 2437912012
1,А1 Болт 1.1 М36х1400 ГОСТ 24379.1-2012 GCC-LGN...,11361400 2437912012
2,Болт 1.2.М12x500 ГОСТ 24379.1-2012 Сборный ГОС...,1212500 2437912012
3,Болт 1.2.МЗ0х900 ГОСТ 24379.1-2012 Сборный ГОС...,1230900 2437912012
4,Болт 2.1.М24х450 ст.09Г2С-6 ГОСТ 24379.1-80 в ...,2124450 0926 2437912012
...,...,...
995,Гайка шестигранная нормальная М16-5 ГОСТ ISO 4...,165 40322014
996,Гайка М12.016 ГОСТ Р ИСО 4032-2014 20 ГОСТ 105...,12016 20 40322014
997,Гайка М12.016 ГОСТ Р ИСО 4032-2014 20 ГОСТ 105...,12016 20 40322014
998,Гайка М12.016 ГОСТ Р ИСО 4032-2014 20 ГОСТ 105...,12016 20 40322014


In [11]:
matches

,original_name,match_name,score,match_index
0,21 248000926 24379180 4 1 1,1124800 092 2437912012,54.761905,0
1,1 11 361400 2437912012 gcclgnddd12510010000k...,11361400 2437912012,12.280702,1
2,1212x500 2437912012 2437902012,2124600 092 2437912012,0.000000,5
3,120900 2437912012 2437902012,1142900 092 2437912012,58.823529,700
4,2124450 0926 24379180 4 1 1,2124450 0926 2437912012,62.500000,4
...,...,...,...,...
995,165 iso 40322014,165 40322014,100.000000,995
996,12016 40322014 20 10502013,12016 20 40322014,33.333333,996
997,12016 40322014 20 10502013,12016 20 40322014,33.333333,996
998,12016 40322014 20 10502013,12016 20 40322014,33.333333,996


In [12]:
combined = pd.merge(train, matches, how='left', left_index=True, right_on='match_index')
selected_columns = ['Описание', 'Эталонная номенклатура', 'score']
combined = combined[selected_columns]



In [13]:
combined.sort_values(by='score', ascending=False).head(1000)


,Описание,Эталонная номенклатура,score
506.0,01 1082756980,01 1082756980,100.000000
269.0,01 341073193,01 341073193,100.000000
468.0,04 341074093,04 341074093,100.000000
471.0,12 341074093,12 341074093,100.000000
984.0,591570 12 121810,12 121810 591570,100.000000
...,...,...,...
90.0,hsar m650,650 hilti hsa,-7.207592
406.0,din 934 iso 4032 m108a280,108 280 din 934,-10.231218
46.0,din 934 iso 4032 m108a280,108 280 din 934,-10.231218
121.0,hus3h 1090,1090 hus3h,-17.322411
